In [37]:
import sys
import pandas as pd
import gensim
import numpy as np

print( F"Python version : {sys.version}\nPandas version : {pd.__version__}\nGensim version : {gensim.__version__}\nNumpy version : {np.__version__}" )

Python version : 3.10.9 (main, Mar  1 2023, 12:33:47) [Clang 14.0.6 ]
Pandas version : 2.0.3
Gensim version : 4.3.0
Numpy version : 1.25.1


In [3]:
df = pd.read_csv("./Car_and_MSRP.csv")
df.tail(3)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920
11913,Lincoln,Zephyr,2006,regular unleaded,221.0,6.0,AUTOMATIC,front wheel drive,4.0,Luxury,Midsize,Sedan,26,17,61,28995


# Clean DATA

In [8]:
df["make_model"] = df["Make"]+" "+df["Model"]

# Select columns which has string values

df_clean = df.loc[:,['Engine Fuel Type','Transmission Type','Driven_Wheels','Market Category','Vehicle Size','Vehicle Style','make_model']]

# Combine all the columns to one column

df_clean = df_clean.apply(lambda x: ','.join( x.astype(str) ), axis=1 )

# convert to dataframe

df_clean = pd.DataFrame({'clean':df_clean})
df_clean.tail(3)

,clean
11911,"premium unleaded (required),AUTOMATIC,all whee..."
11912,"premium unleaded (recommended),AUTOMATIC,all w..."
11913,"regular unleaded,AUTOMATIC,front wheel drive,L..."


In [10]:
sen = [ row.split(',') for row in df_clean.clean ]
sen[:3]

[['premium unleaded (required)',
  'MANUAL',
  'rear wheel drive',
  'Factory Tuner',
  'Luxury',
  'High-Performance',
  'Compact',
  'Coupe',
  'BMW 1 Series M'],
 ['premium unleaded (required)',
  'MANUAL',
  'rear wheel drive',
  'Luxury',
  'Performance',
  'Compact',
  'Convertible',
  'BMW 1 Series'],
 ['premium unleaded (required)',
  'MANUAL',
  'rear wheel drive',
  'Luxury',
  'High-Performance',
  'Compact',
  'Coupe',
  'BMW 1 Series']]

# Finding similarity

In [11]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

In [30]:
model = Word2Vec( sen, min_count=1, vector_size=30, workers=3, window=3, sg=1 )
# size = learning rate, window = number of words before and after, sg = 1 is skip-gram sg = 0 is CBOW (default)

print(model)

Word2Vec<vocab=977, vector_size=30, alpha=0.025>


In [28]:
# model.wv.key_to_index
# The mothod above is to observe a name of cars

In [42]:
# get all vector

model.wv.vectors

array([[ 4.3366980e-02,  6.8079764e-01,  5.8071357e-01, ...,
         2.0558797e-01, -1.1672420e-01, -1.5932523e-01],
       [-4.9585126e-02, -8.2318231e-02,  9.2882150e-01, ...,
        -4.3661472e-01, -1.8378312e-02, -2.1833830e-01],
       [-5.4734570e-01, -2.5774130e-01,  1.8217637e-01, ...,
        -2.6650058e-02, -7.0617443e-01,  2.7539262e-01],
       ...,
       [-5.3690532e-03,  1.4682855e-02,  4.7492664e-02, ...,
         2.8592888e-02, -4.8424017e-02, -4.8213329e-02],
       [-3.2351837e-02,  9.4051277e-03,  2.7724361e-02, ...,
         5.3257751e-04, -5.1531233e-02, -3.6398746e-02],
       [ 3.1328410e-02,  2.3002479e-02,  4.1369472e-02, ...,
         3.1612452e-02, -6.4519355e-03, -4.5937512e-02]], dtype=float32)

In [49]:
# get specific vector

model.wv["Porsche 718 Cayman"]

array([ 0.02758422,  0.04770882,  0.07188908,  0.01238047, -0.00811635,
       -0.00478548, -0.00015456,  0.00301686, -0.03607452, -0.00229742,
        0.05544063, -0.03943737, -0.00861359, -0.03885028, -0.08928543,
        0.00987405,  0.03459644,  0.01833516, -0.05484562,  0.00973742,
        0.04737968, -0.01283314,  0.0012918 ,  0.05715295,  0.01204695,
        0.11842373,  0.02932185,  0.03244522, -0.05436143, -0.06974013],
      dtype=float32)

In [50]:
model.wv.most_similar('Porsche 718 Cayman', topn=10)  # get other similar words

[('BMW M5', 0.9156110286712646),
 ('Toyota MR2', 0.9152314066886902),
 ('Lincoln Mark VII', 0.9134257435798645),
 ('Lexus IS 250 C', 0.9049832224845886),
 ('BMW ALPINA B6 Gran Coupe', 0.9045923948287964),
 ('Chevrolet SS', 0.9006433486938477),
 ('flex-fuel (unleaded/natural gas)', 0.8996391296386719),
 ('Volkswagen Cabrio', 0.8992029428482056),
 ('Hyundai XG350', 0.8975217342376709),
 ('Lexus IS 350 C', 0.8958631157875061)]

In [51]:
# Compare
model.wv.similarity('Porsche 718 Cayman', 'Mercedes-Benz SLK-Class')

0.8220625